# Chapter 3: Data Formats and Loading

**Data-Juicer User Guide**

- Git Commit: `v1.4.6`
- Commit Date: 2026-02-02
- Repository: https://github.com/datajuicer/data-juicer

---

This chapter answers the most common question: "Can my data be used with Data-Juicer, and how do I load it?"

## Table of Contents

1. [Supported File Formats](#supported-file-formats)
2. [DJ Format Specification](#dj-format-specification)
3. [Field Mapping](#field-mapping)
4. [Loading Data](#loading-data)
5. [Format Compatibility Quick Reference](#format-compatibility-quick-reference)
6. [Related Tools](#related-tools)
7. [Further Reading](#further-reading)

In [1]:
# Install Data-Juicer (if not installed)
# If running in Google Colab, use 'pip install' instead of 'uv pip install'
# !uv pip install py-data-juicer

In [2]:
# Setup
import json
import os

os.makedirs('./data', exist_ok=True)
os.makedirs('./configs', exist_ok=True)

samples = [
    {"title": "Introduction to ML", "text": "Machine learning is a subset of AI that enables systems to learn from data."},
    {"title": "Data-Juicer 2.0: Cloud-Scale Adaptive Data Processing for and with Foundation Models", "text": "error text"},
    {"title": "Hi", "text": "This is a long text, which is longer than 20 chars."}
]

with open('./data/sample.jsonl', 'w') as f:
    for s in samples:
        f.write(json.dumps(s) + '\n')

print(f"✅ Created sample.jsonl with {len(samples)} samples")

✅ Created sample.jsonl with 3 samples


---

## Supported File Formats

Data-Juicer natively supports multiple file formats through its formatter system:

| Formatter | Supported Extensions | Description |
|-----------|---------------------|-------------|
| **JsonFormatter** | `.json`, `.jsonl`, `.jsonl.zst` | JSON and JSON Lines (recommended) |
| **ParquetFormatter** | `.parquet` | Apache Parquet (efficient for large datasets) |
| **CsvFormatter** | `.csv` | Comma-separated values |
| **TsvFormatter** | `.tsv` | Tab-separated values |
| **TextFormatter** | `.txt`, `.md`, `.pdf`, `.docx`, code files | Plain text and documents |

### TextFormatter: Extended Support

- **Documents**: `.txt`, `.md`, `.pdf`, `.docx`, `.tex`, `.rst`
- **Code files**: `.py`, `.java`, `.cpp`, `.js`, `.ts`, `.go`, `.rs`, `.rb`, `.php`, `.sql`, `.sh`, `.html`, `.css`, `.xml`, and more

For PDF and DOCX files, Data-Juicer automatically extracts text content.

### Using the YAML Way

In [3]:
# Process using the default 'text' field
config = """project_name: 'basic_demo'
dataset_path: './data/sample.jsonl'
export_path: './outputs/basic_processed.jsonl'

process:
  - text_length_filter:
      min_len: 15   # Filter out samples with text shorter than 15 chars
"""

with open('./configs/basic.yaml', 'w') as f:
    f.write(config)

!dj-process --config ./configs/basic.yaml

2026-02-12 09:22:18.182 | INFO     | data_juicer.config.config:695 - dataset_path config is set and a valid local path
2026-02-12 09:22:18.205 | INFO     | data_juicer.config.config:1012 - Back up the input config file [/workspaces/data-juicer-hub/configs/basic.yaml] into the work_dir [/workspaces/data-juicer-hub/outputs]
2026-02-12 09:22:18.209 | INFO     | data_juicer.config.config:1033 - Configuration table: 
╒══════════════════════════╤═══════════════════════════════════════════════════════════════════════════════╕
│ key                      │ values                                                                        │
╞══════════════════════════╪═══════════════════════════════════════════════════════════════════════════════╡
│ config                   │ [Path_fr(./configs/basic.yaml, cwd=/workspaces/data-juicer-hub)]              │
├──────────────────────────┼───────────────────────────────────────────────────────────────────────────────┤
│ auto                     │ False     

In [4]:
# Check results
print("Processed output (samples with text >= 15 chars):")
with open('./outputs/basic_processed.jsonl', 'r') as f:
    for line in f:
        print(json.loads(line))

Processed output (samples with text >= 15 chars):
{'title': 'Introduction to ML', 'text': 'Machine learning is a subset of AI that enables systems to learn from data.'}
{'title': 'Hi', 'text': 'This is a long text, which is longer than 20 chars.'}


Other formats can be loaded in the same way:

```yaml
# JSONL (most common)
dataset_path: './data/train.jsonl'

# Parquet (efficient for large datasets)
dataset_path: './data/train.parquet'

# CSV
dataset_path: './data/train.csv'

# Specify file suffixes explicitly
dataset_path: './data/'
suffixes: ['.csv', '.json']  # Only load these file types
```

For complex data loading scenarios (data mixing, sampling, remote datasets), see **[Chapter 10: Advanced Dataset Configuration](./10_Advanced_Dataset_Configuration.ipynb)**.

---

## DJ Format Specification

While Data-Juicer supports multiple file formats, the **data structure** follows a unified schema.

### Core Fields

```python
{
  "text": "xxx",           # For pretraining and general language modeling
  "query": "xxx",          # For dialog and question-answering
  "response": "xxx",       # For dialog responses and assistant output
}
```

**Note**: Different dataset types use different core fields. A dialog dataset would have `query` and `response`, while a pretraining dataset would have only `text`.

### Multimodal Fields

For multimodal data, DJ Format uses **file paths** (not embedded data):

```json
{
  "text": "<__dj__image> Antarctica is Earth's southernmost and least-populated continent. <|__dj__eoc|> "
          "<__dj__video> <__dj__audio> Situated almost entirely south of the Antarctic Circle and surrounded by the "
          "Southern Ocean (also known as the Antarctic Ocean), it contains the geographic South Pole. <|__dj__eoc|> "
          "Antarctica is the fifth-largest continent, being about 40% larger than Europe, "
          "and has an area of 14,200,000 km2 (5,500,000 sq mi). <__dj__image> <__dj__image> <|__dj__eoc|> "
          "Most of Antarctica is covered by the Antarctic ice sheet, "
          "with an average thickness of 1.9 km (1.2 mi). <|__dj__eoc|>",
  "images": [
    "path/to/the/image/of/antarctica_snowfield",
    "path/to/the/image/of/antarctica_map",
    "path/to/the/image/of/europe_map"
  ],
  "audios": [
    "path/to/the/audio/of/sound_of_waves_in_Antarctic_Ocean"
  ],
  "videos": [
    "path/to/the/video/of/remote_sensing_view_of_antarctica"
  ]
}
```

**Special Tokens** (configurable in YAML):
- `<__dj__image>`: Image placeholder (config: `image_special_token`)
- `<__dj__audio>`: Audio placeholder (config: `audio_special_token`)
- `<__dj__video>`: Video placeholder (config: `video_special_token`)
- `<|__dj__eoc|>`: End of chunk (config: `eoc_special_token`)

### Metadata Fields

Optional fields for tracking data lineage (which could be primitive or produced by Data-Juicer):

```json
{
  "text": "Sample text...",
  "__dj__meta__": {"source": "wikipedia", "date": "2024-01"},
  "__dj__stats__": {"lang": "en", "text_length": 256}
}
```

---

## Field Mapping

You don't need to rename your fields. Data-Juicer supports field mapping to work with any field names.

### Text Field Mapping (`text_keys`)

If your data uses a different field name for text content:

```yaml
# Your data: {"content": "Hello world", "id": 1}
text_keys: 'content'   # Map 'content' as the text field
```

### Using 'title' Field Instead of 'text'

In [5]:
# Process using 'title' field instead of 'text'
config = """project_name: 'field_mapping_demo'
dataset_path: './data/sample.jsonl'
export_path: './outputs/title_processed.jsonl'

# Field mapping: use 'title' as the text field
text_keys: 'title'

process:
  - text_length_filter:
      min_len: 15   # Filter out samples with title shorter than 15 chars
"""

with open('./configs/field_mapping.yaml', 'w') as f:
    f.write(config)

!dj-process --config ./configs/field_mapping.yaml

2026-02-12 09:22:27.764 | INFO     | data_juicer.config.config:695 - dataset_path config is set and a valid local path
2026-02-12 09:22:27.787 | INFO     | data_juicer.config.config:1012 - Back up the input config file [/workspaces/data-juicer-hub/configs/field_mapping.yaml] into the work_dir [/workspaces/data-juicer-hub/outputs]
2026-02-12 09:22:27.791 | INFO     | data_juicer.config.config:1033 - Configuration table: 
╒══════════════════════════╤═══════════════════════════════════════════════════════════════════════════════╕
│ key                      │ values                                                                        │
╞══════════════════════════╪═══════════════════════════════════════════════════════════════════════════════╡
│ config                   │ [Path_fr(./configs/field_mapping.yaml, cwd=/workspaces/data-juicer-hub)]      │
├──────────────────────────┼───────────────────────────────────────────────────────────────────────────────┤
│ auto                     │ Fa

In [6]:
# Check results
print("Processed output (samples with title >= 15 chars):")
with open('./outputs/title_processed.jsonl', 'r') as f:
    for line in f:
        print(json.loads(line))

Processed output (samples with title >= 15 chars):
{'title': 'Introduction to ML', 'text': 'Machine learning is a subset of AI that enables systems to learn from data.'}
{'title': 'Data-Juicer 2.0: Cloud-Scale Adaptive Data Processing for and with Foundation Models', 'text': 'error text'}


### Multiple Text Fields

When your data has **multiple text fields** that different operators need to process, declare them all in the global config:

```yaml
# Declare ALL text fields upfront
text_keys: ['title', 'text']
```

**Why declare multiple `text_keys` globally?**
- During loading, Data-Juicer validates that all declared fields exist
- Prevents errors when operators access undeclared fields
- If an operator doesn't specify `text_key`, it uses the **first** one in the list

### Different Operators for Different Fields

In [7]:
# Process with different operators for different fields
config = """project_name: 'multi_field_demo'
dataset_path: './data/sample.jsonl'
export_path: './outputs/multi_field_processed.jsonl'

# Declare ALL text fields upfront
text_keys: ['title', 'text']

process:
  - text_length_filter:
      text_key: 'title'    # Process 'title' field
      min_len: 15
  
  - words_num_filter:      # Filter by total word count (>= 5)
      text_key: 'text'     # Process 'text' field  
      min_hum: 5
"""

with open('./configs/multi_field.yaml', 'w') as f:
    f.write(config)

!dj-process --config ./configs/multi_field.yaml

2026-02-12 09:22:37.316 | INFO     | data_juicer.config.config:695 - dataset_path config is set and a valid local path
2026-02-12 09:22:37.351 | INFO     | data_juicer.config.config:1012 - Back up the input config file [/workspaces/data-juicer-hub/configs/multi_field.yaml] into the work_dir [/workspaces/data-juicer-hub/outputs]
2026-02-12 09:22:37.356 | INFO     | data_juicer.config.config:1033 - Configuration table: 
╒══════════════════════════╤═══════════════════════════════════════════════════════════════════════════════╕
│ key                      │ values                                                                        │
╞══════════════════════════╪═══════════════════════════════════════════════════════════════════════════════╡
│ config                   │ [Path_fr(./configs/multi_field.yaml, cwd=/workspaces/data-juicer-hub)]        │
├──────────────────────────┼───────────────────────────────────────────────────────────────────────────────┤
│ auto                     │ Fals

In [8]:
# Check results
with open('./outputs/multi_field_processed.jsonl', 'r') as f:
    for line in f:
        print(json.loads(line))

{'title': 'Introduction to ML', 'text': 'Machine learning is a subset of AI that enables systems to learn from data.'}


### Multimodal Field Mapping

Similarly for images, audio, and video:

```yaml
# Default field names
image_key: 'images'
audio_key: 'audios'
video_key: 'videos'

# Custom mapping:
image_key: 'img_paths'   # Your field is called 'img_paths'
```

---

## More Data Loading Options

### Loading Multiple Files from a Directory

In [9]:
# Create a directory with multiple text files
os.makedirs('./data/multiple_json', exist_ok=True)

multiple_json = [
    ("intro.json", "Machine learning is a powerful technology that enables computers to learn from data."),
    ("deep_learning.json", "Deep learning is a subset of machine learning using neural networks."),
    ("short.json", "Hi.")
]

for filename, content in multiple_json:
    data = {"text": content}
    with open(f'./data/multiple_json/{filename}', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

print("✅ Created text files:")
for filename, content in multiple_json:
    print(f"  - {filename}: '{content[:40]}...'" if len(content) > 40 else f"  - {filename}: '{content}'")

✅ Created text files:
  - intro.json: 'Machine learning is a powerful technolog...'
  - deep_learning.json: 'Deep learning is a subset of machine lea...'
  - short.json: 'Hi.'


In [10]:
# Load all .txt files from directory
config = """project_name: 'directory_demo'
dataset_path: './data/multiple_json/'

suffixes: ['.json']

export_path: './outputs/json_processed.jsonl'

process:
  - text_length_filter:
      min_len: 20
"""

with open('./configs/directory.yaml', 'w') as f:
    f.write(config)

!dj-process --config ./configs/directory.yaml

2026-02-12 09:22:48.044 | INFO     | data_juicer.config.config:695 - dataset_path config is set and a valid local path
2026-02-12 09:22:48.068 | INFO     | data_juicer.config.config:1012 - Back up the input config file [/workspaces/data-juicer-hub/configs/directory.yaml] into the work_dir [/workspaces/data-juicer-hub/outputs]
2026-02-12 09:22:48.073 | INFO     | data_juicer.config.config:1033 - Configuration table: 
╒══════════════════════════╤═══════════════════════════════════════════════════════════════════════════════╕
│ key                      │ values                                                                        │
╞══════════════════════════╪═══════════════════════════════════════════════════════════════════════════════╡
│ config                   │ [Path_fr(./configs/directory.yaml, cwd=/workspaces/data-juicer-hub)]          │
├──────────────────────────┼───────────────────────────────────────────────────────────────────────────────┤
│ auto                     │ False 

In [11]:
# Check results - "Hi." should be filtered out
print("Processed output (text files with >= 20 chars):")
with open('./outputs/json_processed.jsonl', 'r') as f:
    for line in f:
        data = json.loads(line)
        print(f"  - {data['text'][:60]}..." if len(data['text']) > 60 else f"  - {data['text']}")

Processed output (text files with >= 20 chars):
  - Machine learning is a powerful technology that enables compu...
  - Deep learning is a subset of machine learning using neural n...


### HuggingFace Hub

Load datasets directly from HuggingFace Hub:

```yaml
dataset:
  configs:
    - type: 'remote'
      source: 'huggingface'
      path: "HuggingFaceFW/fineweb"
      name: "CC-MAIN-2024-10"
      split: "train"
      limit: 1000
```


### Cloud Storage (S3)

Load data from S3-compatible storage:

```yaml
# Load all JSON files from an S3 directory
dataset:
  path: s3://my-bucket/data/json-files/
  format: json  # Must specify format for directory paths
  aws_access_key_id: xxx
  aws_secret_access_key: xxx

# Load all Parquet files from an S3 directory
dataset:
  path: s3://my-bucket/data/parquet-files/
  format: parquet
  aws_access_key_id: xxx
  aws_secret_access_key: xxx
```

For more data loading configurations, please refer to [Dataset Configuration](https://datajuicer.github.io/data-juicer/en/main/docs/DatasetCfg.html).

---

## Format Compatibility Quick Reference

| Your Data Format | Can Use Directly? | Action Needed |
|------------------|-------------------|---------------|
| JSONL with `text` field | ✅ Yes | None |
| JSONL with other field names | ✅ Yes | Set `text_keys: 'your_field'` |
| Parquet / CSV / TSV | ✅ Yes | Set `text_keys` if needed |
| Plain text files (.txt, .md) | ✅ Yes | Each file → one sample |
| PDF / DOCX | ✅ Yes | Text auto-extracted |
| ShareGPT format | ❌ Convert first | Use [`llama_factory_sharegpt_to_dj.py`](https://github.com/datajuicer/data-juicer/blob/main/tools/fmt_conversion/post_tuning_dialog/source_format_to_data_juicer_format/llama_factory_sharegpt_to_dj.py) [`ms_swift_sharegpt_to_dj.py`](https://github.com/datajuicer/data-juicer/blob/main/tools/fmt_conversion/post_tuning_dialog/source_format_to_data_juicer_format/ms_swift_sharegpt_to_dj.py)|
| Alpaca format | ❌ Convert first | Use [`alpaca_to_dj.py`](https://github.com/datajuicer/data-juicer/blob/main/tools/fmt_conversion/post_tuning_dialog/source_format_to_data_juicer_format/alpaca_to_dj.py) |
| Messages format (OpenAI-style) | ❌ Convert first | Use [`messages_to_dj.py`](https://github.com/datajuicer/data-juicer/blob/main/tools/fmt_conversion/post_tuning_dialog/source_format_to_data_juicer_format/messages_to_dj.py) |
| LLaVA (image-text) | ❌ Convert first | Use [`llava_to_dj.py`](https://github.com/datajuicer/data-juicer/blob/main/tools/fmt_conversion/multimodal/source_format_to_data_juicer_format/llava_to_dj.py) |

---

## Related Tools

Data-Juicer provides additional tools for data preparation:

| Tool Category | Description | Documentation | Notebook |
|---------------|-------------|---------------|-----------|
| **Format Conversion** | Convert between formats (ShareGPT, Alpaca, LLaVA, etc.) | [📖 post_tuning_dialog](https://datajuicer.github.io/data-juicer/en/main/tools/fmt_conversion/post_tuning_dialog/README.html) [📖 Multimodal](https://datajuicer.github.io/data-juicer/en/main/tools/fmt_conversion/multimodal/README.html) | [Chapter 9: Multimodal Data Processing](./09_Multimodal_Data_Processing.ipynb) |
| **Preprocessing** | Prepare raw data before processing | [📖 preprocess](https://datajuicer.github.io/data-juicer/en/main/tools/preprocess/README.html) | [Chapter 8: Preprocessing](./08_Preprocessing.ipynb) |
| **Postprocessing** | Transform processed data for downstream tasks | [📖 postprocess](https://datajuicer.github.io/data-juicer/en/main/tools/postprocess/README.html) | \ |

These tools are located in `tools/` directory of the Data-Juicer repository.

---

## Further Reading

- 📖 [Dataset Configuration](https://datajuicer.github.io/data-juicer/en/main/docs/DatasetCfg.html)
- 📖 [Configuration Reference](https://github.com/datajuicer/data-juicer/blob/main/data_juicer/config/config_all.yaml)
- 📖 [Format Conversion Documentation](https://datajuicer.github.io/data-juicer/en/main/tools/fmt_conversion/README.html)